In [ ]:
import re
import pandas as pd
import numpy as np

### 데이터 피쳐 설명
    
    "region", # 지역
    "cpname", # 회사이름
    "title", # 구인공고 제목
    "career", # 요구 경력
    "education", # 요구 학력
    "jobtype", # 고용 형태 (정규직, 비정규직, ...)
    "cptype", # 회사 규모 형태 (대기업, 중견, 중소, ...)
    "sales", # 회사 연매출
    "employees", # 회사 근무 직원 수 
    "aversalary", # 회사 근무 직원 평균 연봉
    "capital", # 회사 자본금
    "pros", # 인크루트에 등록된 '입사지원하면 좋은 이유' 들

In [ ]:
# 17개 지역
regions = {
    "11": "서울",
    "12": "부산",
    "14": "인천",
    "16": "대전",
    "15": "광주",
    "13": "대구",
    "17": "울산",
    "27": "세종",
    "18": "경기",
    "19": "강원특별자치도",
    "20": "충북",
    "21": "충남",
    "22": "전북특별자치도",
    "23": "전남",
    "24": "경북",
    "25": "경남",
    "26": "제주",
}


# 수도권 : 서울, 인천, 세종, 경기
# 수도권 외 광역시: 부산, 대구, 광주, 대전, 울산
# 그 외 지방: 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

features = [
    "region", # 지역
    "cpname", # 회사이름
    "title", # 구인공고 제목
    "career", # 요구 경력
    "education", # 요구 학력
    "jobtype", # 고용 형태 (정규직, 비정규직, ...)
    "cptype", # 회사 규모 형태 (대기업, 중견, 중소, ...)
    "sales", # 회사 연매출
    "employees", # 회사 근무 직원 수 
    "aversalary", # 회사 근무 직원 평균 연봉
    "capital", # 회사 자본금
    "pros", # 인크루트에 등록된 '입사지원하면 좋은 이유' 들
]
df = pd.DataFrame(columns=features)

for region in regions.values():
    df_subset = pd.read_csv(f"sample data/raw_data_{region}.csv", index_col=0)
    df = pd.concat([df, df_subset])

df = df.reset_index(drop=True)

df.to_csv("raw_data.csv", encoding="utf-8-sig")

In [ ]:
df = pd.read_csv('raw_data.csv', encoding='utf-8-sig', index_col=0)

In [ ]:
df.drop('cpname', axis=1, inplace=True)
df.drop('title', axis=1, inplace=True)
# 'career' 항목은 데이터 크롤링 과정에서 신입/연차 무관 으로 사전에 필터링 되었으므로 필요없다.
df.drop('career', axis=1, inplace=True)
df.head()

In [ ]:
print(df['education'].value_counts())
print(df['jobtype'].value_counts())
print(df['cptype'].value_counts())

### 인코딩

범주에 따라 다른 점수를 부여하고 싶어서 범주형 데이터에 대해 수동으로 라벨 인코딩 수행  

In [ ]:
df['education'] = df['education'].replace({
    '학력무관': 0,
    '고졸': 1,
    '고졸이상': 1,
    '초대졸' : 2,
    '초대졸이상': 2,
    '대졸': 3,
    '대졸이상': 3,
    '석사' : 4,
    '석사이상' : 4,
    '박사' : 5,
    '박사이상' : 5
},)

def encode_cptype(x : str):
    if x == '대기업':
        return 2
    elif x == '중견기업':
        return 1
    else:
        return 0
    

df['jobtype'] = df['jobtype'].map(lambda x : 1 if x == '정규직' else 0)

df['cptype'] = df['cptype'].map(encode_cptype)

In [ ]:
print(df['education'].value_counts())
print(df['jobtype'].value_counts())
print(df['cptype'].value_counts())
df.head()

### 인코딩 
수집한 지역을 {수도권, 비수도권 광역시, 그 외 지방} 중 하나로 분류 및 숫자값으로 라벨 인코딩  

In [ ]:
# 수도권 : 서울, 인천, 세종, 경기
# 수도권 외 광역시: 부산, 대구, 광주, 대전, 울산
# 그 외 지방: 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

def encode_region(x : str):
    if x == '서울' or x =='인천' or x =='세종' or x =='경기':
        return 2
    elif x == '부산' or x == '대구'or x == '광주'or x =='대전' or x =='울산':
        return 1
    else:
        return 0
    

df['region'] = df['region'].map(encode_region)

In [ ]:
print(df['region'].value_counts())

### 인코딩  
'pros' 피쳐는 문자열 값이므로 이에 대한 처리 수행

'pros' 값을 점수로 변환하여 'pros_encoded' 피쳐로 변환하여 처리  
이에 대한 점수 기준표는 tags_encoded.csv 파일에 등록.  
점수로 변환하기 이전 'pros' 문자열 값들은 tags.csv 파일에 등록되어있다.  

'매출 1000대 기업', '대기업', 'KOSPI 상장' 등 객관적인 평가지표가 반영된 항목이면서 기업역량이 우수함을 쉽게 판단할 수 있는 항목에 주로 5점,
'육아휴직 제공', '자기 계발비 지원' 등 해당 항목이 부여된 기준을 알기 어렵고, 사실을 검증하기 어려운 항목들에 대해서는 1점을 주는 식으로 평가하였다.

In [ ]:
tags_encoded = pd.read_csv('tags_encoded.csv', encoding='utf-8-sig', index_col=0)
# tags_encoded.head()
tags_score = dict(zip(tags_encoded['tags'], tags_encoded['score']))
tags_score

In [ ]:
def encodeTagsByScore(tags: str):
    if isinstance(tags, float): return tags
    total_score = 0
    for tag in tags.split('-'):
        if tag !=  '':
            total_score += tags_score[tag]
    return total_score

df['pros_encoded'] = df['pros'].apply(encodeTagsByScore)
df['pros_encoded']

### 데이터 형변환  
문자열 값으로 수집된 데이터 실수형 데이터로 변환  

In [ ]:
# 띄어쓰기, 콤마(',') 제거
for col in ['sales', 'aversalary', 'capital', 'employees']:
    df[col] = df[col].map(lambda x : x[:x.rfind('(')] if x.rfind('(') != -1 else x)
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace(' ', '')
    

# 빈 값이라면 '-' 으로 대체
df['sales'] = df['sales'].apply(lambda x : x if x != '' else '-')
df['aversalary'] = df['aversalary'].str.replace('\n자세히보기', '')
df['employees'] = df['employees'].map(lambda x : x[:x.rfind('명')])

In [ ]:
nullable_features = ['sales',	'employees',	'aversalary',	'capital']

# '-' 값들 모두 np.nan 로 변경
for nullable_feature in nullable_features:
    # 값이 음수 (sales : - 15억 3,272만원 (2023) ) 인 경우도 있으므로 replace 함수가 아닌 map 함수 사용
    df[nullable_feature] = df[nullable_feature].map(lambda x : np.nan if x == '-' else x)

# np.nan 값 0 으로 변경
df['pros_encoded'] = df['pros_encoded'].replace(np.nan, 0)

# np.nan 값을 제외하고 float 로 형변환. (np.nan 는 float)
# 5월 4일 기준으로 수정(line 242)했지만, 수정 이전에는 crawling.py 파일에서 employees 의 결측값을 empty string 으로 저장
# 이미 크롤링한 raw_data.csv 파일 기준으로는 바로 아래 코드가 필요
df['employees'] = df['employees'].map(lambda x: float(x) if x != '' else np.nan)
# 수정된 버전
# df['employees'] = df['employees'].map(lambda x: float(x))

print(df['sales'].isna().sum())
print(df['employees'].isna().sum())
print(df['aversalary'].isna().sum())
print(df['capital'].isna().sum())
print(df['pros_encoded'].isna().sum())

print(df.dtypes)
print(type(df['sales'][0]))
print(type(df['aversalary'][0]))
print(type(df['capital'][0]))

df.head()

In [ ]:
def convertFromStr(input):
    # 단위 : 백만원
    # 띄어쓰기가 없다고 전제
    # input = input.replace(' ', '')
    # input : str or np.nan
    # type(np.nan) == float
    if type(input) != str: return input
    p = re.compile('(\d{1,4}조)?(\d{1,4}억)?(\d{1,4}만)?원')
    match = p.search(input)
    if (match is None) : 
        print(input) # '-'
        return input
        
    value = 0
    for group in match.groups():
        if group is None: # 마지막 group 은 None
            continue
        elif group[-1] == '조':
            value += int(group[:-1]) * 1000000
        elif group[-1] == '억':
            value += int(group[:-1]) * 100
        elif group[-1] == '만':
            value += int(group[:-1]) // 100
        else: 
            continue
    return value

for col in ['sales', 'aversalary', 'capital', 'capital']:
    # 단위 : 백만원
    df[col] = df[col].map(convertFromStr)

In [ ]:
df.drop('pros', axis=1, inplace=True)
print(df.dtypes)
df.head()

In [ ]:
# 피쳐 스케일링 이전 값 저장
df.to_csv('data_beforeScaling.csv', encoding='utf-8-sig')